In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from catboost import CatBoostClassifier, Pool, cv

import warnings
warnings.filterwarnings("ignore")

/home/alan/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Загрузка датасета и просмотр основной по нему информации**

In [2]:
df = pd.read_csv("surveys.csv")

In [3]:
new_names = [
    'Отрасль',
    'Направление',
    'Бюджет',
    'Время на внедрение',
    'Миссия',
    'Цели',
    'Риски',
    'Ресурсы',
    'Бизнес-модель',
    'Нормативно-правовая база',
    'Охват',
    'Структура собственности',
    'Конкурентная среда',
    'Целевой рынок',
    'Ценовая стратегия',
    'Основной продукт',
    'Каналы сбыта',
    'Доствка',
    'Количество сотрудников',
    'Обучение персонала',
    'Инструмент коммуникации',
    'Инструменты совместной работы',
    'Работа вне офиса',
    'Уровень автоматизации',
    'Маркетинговая стратегия',
    'Бренд-стратегия',
    'Стратегия удержания клиентов',
    'Платформа социальных сетей',
    'Функциональные возможности',
    'Соответ. арх. пред.',
    'Бизнес-процессы',
    'Аппаратное обеспечение',
    'Программное обеспечение',
    'Безопасность',
    'Резервное копирование',
    'Масштабируемость',
    'Интеграция',
    'Техническая поддержка',
    'Гарантийные обязательства',
    'Отчётность',
    'Эксплуатационная документация',
    'Аналитика и прогнозирование',
    'Управление данными',
    'Пользовательский интерфейс',
    'Открытое ПО', 
    'Класс'
]

In [4]:
df.columns = new_names

In [5]:
checkbox_questions = [
    'Риски', 'Ресурсы', 'Каналы сбыта', 'Маркетинговая стратегия', 'Функциональные возможности',
    'Бизнес-процессы', 'Аппаратное обеспечение', 'Программное обеспечение', 'Безопасность',
    'Масштабируемость', 'Интеграция', 'Техническая поддержка', 'Гарантийные обязательства',
    'Отчётность', 'Эксплуатационная документация', 'Аналитика и прогнозирование',
    'Управление данными', 'Пользовательский интерфейс'
]

In [6]:
df.head()

,Отрасль,Направление,Бюджет,Время на внедрение,Миссия,Цели,Риски,Ресурсы,Бизнес-модель,Нормативно-правовая база,...,Интеграция,Техническая поддержка,Гарантийные обязательства,Отчётность,Эксплуатационная документация,Аналитика и прогнозирование,Управление данными,Пользовательский интерфейс,Открытое ПО,Класс
0,Финансы и страхование,Розничная торговля,от 10 до 20 процентов от общего бюджета,от 6 месяцев до 1 года,Стремление к лидерству в отрасли и расширение ...,Расширение ассортимента товаров и услуг.,"Экономические риски, Технологические риски, Ри...","Информационные ресурсы (например, базы данных)...",Определение целевой аудитории и ее потребностей.,Строго регулируемая,...,"Интеграция с бухгалтерскими системами, Интегра...","Техническая поддержка 24/7, Ответы в течение о...","Гарантия на аппаратное обеспечение, Гарантия н...",Предоставление стандартных отчетов по продажам...,Предоставление технической документации на рус...,"Возможность прогнозирования продаж и доходов, ...",Возможность импорта данных из других систем (E...,"Совместимость с различными устройствами, Совме...","С некоторым опасением, не уверен, что это дост...",Финансовый
1,Производство,Предоставление услуг,от 10 до 20 процентов от общего бюджета,от 3 до 6 месяцев,Инновации и внедрение новых технологий для улу...,Улучшение качества обслуживания клиентов.,"Риски связанные с персоналом, Риски связанные ...","Материальные ресурсы (например, оборудование)",Определение целевой аудитории и ее потребностей.,Умеренно регулируемая,...,"Интеграция с бухгалтерскими системами, Интегра...",Ответы в течение определенного времени,"Гарантия на аппаратное обеспечение, Гарантия н...",Предоставление стандартных отчетов по продажам...,Предоставление информации о стандартах и норма...,Возможность анализа эффективности маркетинговы...,"Хранение данных, Возможность очистки и дедубли...","Интуитивно понятный интерфейс, Многоязычный ин...",Нейтрально,Доставка и хранение
2,Торговля,Розничная торговля,от 20 до 30 процентов от общего бюджета,от 6 месяцев до 1 года,Стремление к лидерству в отрасли и расширение ...,Расширение ассортимента товаров и услуг.,"Экономические риски, Технологические риски, Ри...","Материальные ресурсы (например, оборудование),...",Определение целевой аудитории и ее потребностей.,Умеренно регулируемая,...,"Интеграция с бухгалтерскими системами, Интегра...","Техническая поддержка 24/7, Ответы в течение о...","Гарантия на аппаратное обеспечение, Гарантия н...",Предоставление стандартных отчетов по продажам...,Предоставление технической документации на рус...,Предоставление стандартных аналитических отчет...,"Хранение данных, Возможность очистки и дедубли...","Интуитивно понятный интерфейс, Настраиваемый и...","В целом положительно, но зависит от конкретног...",Торговый
3,Торговля,Розничная торговля,от 20 до 30 процентов от общего бюджета,от 6 месяцев до 1 года,Стремление к лидерству в отрасли и расширение ...,Расширение ассортимента товаров и услуг.,"Экономические риски, Технологические риски, Ри...","Материальные ресурсы (например, оборудование),...",Определение целевой аудитории и ее потребностей.,Умеренно регулируемая,...,"Интеграция с бухгалтерскими системами, Интегра...","Техническая поддержка 24/7, Ответы в течение о...","Гарантия на аппаратное обеспечение, Гарантия н...",Предоставление стандартных отчетов по продажам...,Предоставление технической документации на рус...,Предоставление стандартных аналитических отчет...,"Хранение данных, Возможность очистки и дедубли...","Интуитивно понятный интерфейс, Настраиваемый и...","В целом положительно, но зависит от конкретног...",Торговый
4,Торговля,Розничная торговля,от 20 до 30 процентов от общего бюджета,от 6 месяцев до 1 года,Инновации и внедрение новых технологий для улу...,Расширение ассортимента товаров и услуг.,"Экономические риски, Технологические риски, Ри...","Материальные ресурсы (например, оборудование),...",Определение целевой аудитории и ее потребностей.,Строго регулируемая,...,"Интеграция с бухгалтерским

In [7]:
df.describe()

,Отрасль,Направление,Бюджет,Время на внедрение,Миссия,Цели,Риски,Ресурсы,Бизнес-модель,Нормативно-правовая база,...,Интеграция,Техническая поддержка,Гарантийные обязательства,Отчётность,Эксплуатационная документация,Аналитика и прогнозирование,Управление данными,Пользовательский интерфейс,Открытое ПО,Класс
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,4,4,3,2,2,2,4,8,1,2,...,3,5,5,8,5,8,8,7,5,8
top,Производство,Розничная торговля,от 10 до 20 процентов от общего бюджета,от 6 месяцев до 1 года,Инновации и внедрение новых технологий для улу...,Расширение ассортимента товаров и услуг.,"Экономические риски, Технологические риски, Ри...","Материальные ресурсы (например, оборудование),...",Определение целевой аудитории и ее потребностей.,Умеренно регулируемая,...,"Интеграция с бухгалтерскими системами, Интегра...","Техническая поддержка 24/7, Ответы в течение о...","Гарантия на аппаратное обеспечение, Гарантия н...",Предоставление стандартных отчетов по продажам...,Предоставление технической документации на рус...,Предоставление стандартных аналитических отчет...,"Хранение данных, Возможность очистки и дедубли...","Интуитивно понятный интерфейс, Настраиваемый и...","В целом положительно, но зависит от конкретног...",Торговый
freq,3688,3977,5176,7017,7284,5924,5712,2389,10000,8114,...,5897,2456,4661,2387,3724,2399,2394,3620,5467,2398


In [8]:
def statistics_by_values(DF):
    """
    Напишем функцию для вывода краткой статистики по датасету, включающей информацию о:
    1) уникальных значениях;
    2) нулевых значениях;
    3) пустых значениях;
    4) процентном соотношении для каждого указанного параметра от общего числа данных.
    """
    num_of_unique = DF.nunique()
    num_of_nulls = np.array([])
    for i in DF.columns:
        num_of_nulls = np.append(num_of_nulls, (DF[i] == 0).sum())
    num_of_nans = DF.isna().sum()
    data = {'Уникальные значения': num_of_unique,
            'Процент уникальных значений': num_of_unique/len(DF)*100,
            'Нулевые значения': num_of_nulls, 
            'Процент нулевых значений': num_of_nulls/len(DF)*100,
            'Пустые значения': num_of_nans,
            'Процент пустых значений': num_of_nans/len(DF)*100}
    interesting_values = pd.DataFrame(data, index=DF.columns)
    return interesting_values

In [9]:
statistics_by_values(df)

,Уникальные значения,Процент уникальных значений,Нулевые значения,Процент нулевых значений,Пустые значения,Процент пустых значений
Отрасль,4,0.04,0.0,0.0,0,0.0
Направление,4,0.04,0.0,0.0,0,0.0
Бюджет,3,0.03,0.0,0.0,0,0.0
Время на внедрение,2,0.02,0.0,0.0,0,0.0
Миссия,2,0.02,0.0,0.0,0,0.0
Цели,2,0.02,0.0,0.0,0,0.0
Риски,4,0.04,0.0,0.0,0,0.0
Ресурсы,8,0.08,0.0,0.0,0,0.0
Бизнес-модель,1,0.01,0.0,0.0,0,0.0
Нормативно-правовая база,2,0.02,0.0,0.0,0,0.0


## **Преобразование целевой переменной**

In [10]:
# определение целевой переменной
y = df[["Класс"]]
df = df.drop('Класс', axis=1)

In [11]:
# кодирование целевой переменной
y_dict = {'Государственный' : 1,
          'Социально-ориентированный' : 2,
          'Инновационно-исследовательский' : 3,
          'Сервисный' : 4,
          'Доставка и хранение' : 5,
          'Финансовый' : 6,
          'Производстенный' : 7,
          'Торговый' : 8}

y = y.replace(y_dict)

## **Преобразование датасета - использование Label Encoder для вопросов с несколькими вариантами ответов**

In [12]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns 

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        """
        Преобразует столбцы X, указанные в self.columns, используя
        LabelEncoder(). Если столбцы не указаны, преобразует все
        столбцы в Х.
        """

        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [13]:
# преобразование стринг формата вопросов с несколькими вариантами ответов в список
for col in checkbox_questions:
    df[col] = df[col].str.split(',')

In [14]:
# one-hot-encoding вопросов с несколькими вариантами ответов
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)
for i in checkbox_questions:
    df = df.join(
                pd.DataFrame.sparse.from_spmatrix(
                    mlb.fit_transform(df.pop(i)),
                    index=df.index,
                    columns=[i+"_"+str(col) for col in mlb.classes_]))

In [15]:
# возвращение целевой переменной в обработанный датасет
X = df
df = pd.concat([df, y], axis=1)

## **Обучение модели классификации с использованием библиотеки CatBoost**

In [16]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение категориальных переменных
cat_features = list(X.select_dtypes(include=['object']).columns)

# Создание объектов Pool для обучающей и тестовой выборок
train_data = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_data = Pool(data=X_test, label=y_test, cat_features=cat_features)

In [17]:
# Определение сетки параметров для поиска
grid = {'iterations': [100],
        'learning_rate': [0.1, 0.3, 0.5, 0.7],
        'depth': [2, 3, 4, 5],
        'l2_leaf_reg': [3, 5, 7]}

# Инициализация модели CatBoostClassifier
model = CatBoostClassifier(loss_function='MultiClass', logging_level='Silent')

In [18]:
# Проведение поиска по сетке для нахождения оптимальных гиперпараметров модели
grid_search_result = model.grid_search(grid,
                                       X=train_data,
                                       cv=5,
                                       partition_random_seed=42,
                                       calc_cv_statistics=True,
                                       search_by_train_test_split=True,
                                       shuffle=True,
                                       stratified=True,
                                       train_size=0.8,
                                       verbose=False
                                      )

In [19]:
# Вывод лучших гиперпараметров
print(f"Best parameters: {grid_search_result['params']}")

Best parameters: {'depth': 4, 'l2_leaf_reg': 5, 'iterations': 100, 'learning_rate': 0.1}


In [20]:
# Обучение модели с использованием оптимальных гиперпараметров на обучающих данных
model = CatBoostClassifier(iterations=grid_search_result['params']['iterations'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           loss_function='MultiClass',
                           verbose=False)
model.fit(train_data)

In [21]:
# Предсказывание результатов на тестовой выборке
preds = model.predict(test_data)

In [22]:
# Оценка точности результатов на тестовой выборке (метрика accuracy)
accuracy = np.sum(preds == y_test) / len(y_test)
print(f"Точность: {accuracy_score(preds, y_test)}")

Точность: 0.9345
